In [1]:
import os
from nltk.corpus import stopwords
import string
import pandas as pd

In [2]:
STOPS = stopwords.words("english")
EXCLUDE = set(string.punctuation) | set(["''", "BR", "--", "/td", "nbsp", "2002", "localhost", "&nbsp"])
SPAMPATH = "./spam/"
HAMPATH = "./easy_ham/"

In [3]:
df = pd.read_csv("./quest2020.csv")
df.head()

,id,interview,interview_score,in_quest,gender,race,credits,gpa,school,honors,all_answers
0,2952,0,NaN,0,m,Asian,47,4.00,Smith,1,My ability to efficiently allocate resources i...
1,3025,0,NaN,0,f,Asian,14,3.67,Smith,1,Volunteering at Terps for Change and now new s...
2,2961,1,3.5,1,f,Asian,15,4.00,Smith,1,Improving as a dancer and making the Maryland ...
3,3041,1,4.0,1,f,Asian,14,3.81,Smith,1,I restructured tennis uniforms for inclusivity...
4,3113,0,NaN,0,m,White,16,3.83,Clark,1,My ability to adapt to different environments....


In [4]:
QUESTWORDS = ['quest', 'QUEST', 'Quest']

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [5]:
df['wordcount']=0
df['questcount']=0

# other possibilities:
# word_count
# teamwork_count
# grade_level
# adjective_count
# adverb_count
# sentiment
# quest_count
# quest_upper

for ind in df.index:
    li = list(df.loc[ind, 'all_answers'].split(' '))
    df.loc[ind, 'wordcount'] = len(li)
    df.loc[ind, 'questcount'] = len(intersection(li, QUESTWORDS)) #this is not yet right

df.head()

,id,interview,interview_score,in_quest,gender,race,credits,gpa,school,honors,all_answers,wordcount,questcount
0,2952,0,NaN,0,m,Asian,47,4.00,Smith,1,My ability to efficiently allocate resources i...,1101,1
1,3025,0,NaN,0,f,Asian,14,3.67,Smith,1,Volunteering at Terps for Change and now new s...,890,1
2,2961,1,3.5,1,f,Asian,15,4.00,Smith,1,Improving as a dancer and making the Maryland ...,1185,1
3,3041,1,4.0,1,f,Asian,14,3.81,Smith,1,I restructured tennis uniforms for inclusivity...,927,1
4,3113,0,NaN,0,m,White,16,3.83,Clark,1,My ability to adapt to different environments....,638,1
